In [1]:
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

# This line will hide code by default when the notebook is exported as HTML
# di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

display(HTML("<style>.container { width:100% !important; }</style>"))

import hail as hl
hl.init()

Running on Apache Spark version 2.4.3
SparkUI available at http://p0440.prometheus:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.30-2ae07d872f43
LOGGING: writing to /net/archive/groups/plggneuromol/GTS-analysis/analysis/hail-20200909-2034-0.2.30-2ae07d872f43.log


In [2]:
from hail.plot import show
from pprint import pprint
from bokeh.layouts import gridplot
hl.plot.output_notebook()

import numpy as np
import pandas as pd
from functools import reduce
from itertools import chain
import statistics as stat

Loading BokehJS ...

In [3]:
from bokeh.plotting import figure, show, output_notebook
output_notebook()

Loading BokehJS ...

In [4]:
# Install a pip package in the current Jupyter kernel :)
import sys
!{sys.executable} -m pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/net/software/local/python/3.6.5/bin/python3.6 -m pip install --upgrade pip' command.


In [4]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS.mt')

In [5]:
mt.count()

(16233065, 185)

In [6]:
pheno = hl.import_table('/net/archive/groups/plggneuromol/GTS-analysis/GTS-coded.csv', delimiter = '\t', key='ID')

2020-09-09 20:34:38 Hail: INFO: Reading table with no type imputation
  Loading column 'ID' as type 'str' (type not specified)
  Loading column 'family' as type 'str' (type not specified)
  Loading column 'sex' as type 'str' (type not specified)
  Loading column 'kinship' as type 'str' (type not specified)
  Loading column 'disease' as type 'str' (type not specified)
  Loading column 'phenotype' as type 'str' (type not specified)
  Loading column 'add_pheno' as type 'str' (type not specified)
  Loading column 'heavy_tics' as type 'str' (type not specified)



In [7]:
mt = mt.annotate_cols(phenotypes = pheno[mt.s])

In [8]:
mt.write('/net/archive/groups/plggneuromol/GTS-analysis/data/GTS-with-pheno.mt')

2020-09-09 20:36:08 Hail: INFO: wrote matrix table with 16233065 rows and 185 columns in 1604 partitions to /net/archive/groups/plggneuromol/GTS-analysis/data/GTS-with-pheno.mt


In [156]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/GTS-with-pheno.mt')

In [10]:
mt = mt.annotate_rows(cases_with_gts = hl.agg.group_by(mt.phenotypes.family, hl.agg.count_where(mt.phenotypes.disease == 'YES')),
                      variants_with_gts = hl.agg.group_by((mt.phenotypes.family, mt.phenotypes.disease), hl.agg.count_where(mt.GT.is_non_ref())),
                      carriers_in_families = hl.agg.group_by(mt.s, hl.agg.count_where(mt.GT.is_non_ref())),
                      variants_in_sporadic_cases = hl.agg.group_by((mt.phenotypes.family, mt.phenotypes.disease), hl.agg.counter(mt.GT.is_non_ref()))
                     )

In [11]:
mt = mt.annotate_rows(sporadic_cases = hl.str(mt.variants_in_sporadic_cases[('.','YES')]))

In [12]:
mt = mt.annotate_rows(variants_in_familial = hl.agg.group_by((mt.phenotypes.disease), hl.agg.counter(mt.GT.is_non_ref())))

In [13]:
mt = mt.annotate_rows(variants_in_familial_controls = hl.str(mt.variants_in_familial['NO']),
                      variants_in_familial_cases = hl.str(mt.variants_in_familial['YES']))

In [18]:
mt = mt.filter_rows((mt.gnomad_v3.v3_nfe.AF < 0.01))
mt = mt.filter_cols((mt.phenotypes.family != '.') & (hl.is_defined(mt.phenotypes.family)))
mt = mt.filter_rows(hl.agg.any(mt.GT.is_non_ref()))

In [157]:
sample_dict = hl.import_table('/net/archive/groups/plggneuromol/GTS-analysis/gts-sample-dictionary.csv', delimiter = '\t', key='intelliseq ID', impute = True)

2020-08-10 15:23:12 Hail: INFO: Reading table to impute column types
2020-08-10 15:25:02 Hail: INFO: Finished type imputation
  Loading column 'intelliseq ID' as type 'str' (imputed)
  Loading column 'barcode' as type 'str' (imputed)


In [158]:
mt = mt.annotate_cols(alt_id = sample_dict[mt.s])

In [159]:
fams = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'R', 'S', 'T', 'U', 'W', 'X1', 'X2', 'Y']

In [160]:
len(fams)

18

In [161]:
mtxs = []

A = mt.filter_cols(mt.phenotypes.family == 'A')
A = A.filter_rows((hl.agg.any(A.GT.is_non_ref())) &
                    ((A.variants_with_gts[('A','YES')]) == (A.cases_with_gts['A'])) &
                    (A.carriers_in_families['S_7213'] == 0))


mtxs.append(A)


B = mt.filter_cols(mt.phenotypes.family == 'B')
B = B.filter_rows((hl.agg.any(B.GT.is_non_ref())) &
                    ((B.variants_with_gts[('B','YES')]) == (B.cases_with_gts['B'])) &
                    (B.carriers_in_families['S_7227'] == 0) &
                    (B.carriers_in_families['S_7255'] == 1))

mtxs.append(B)


C = mt.filter_cols(mt.phenotypes.family == 'C')
C = C.filter_rows((hl.agg.any(C.GT.is_non_ref())) &
                    ((C.variants_with_gts[('C','YES')]) == (C.cases_with_gts['C'])) &
                    (C.carriers_in_families['S_7241'] == 0))

mtxs.append(C)


D = mt.filter_cols(mt.phenotypes.family == 'D')
D = D.filter_rows((hl.agg.any(D.GT.is_non_ref())) &
                    ((D.variants_with_gts[('D','YES')]) == (D.cases_with_gts['D'])) &
                    (D.carriers_in_families['S_7246'] == 0) &
                    (D.carriers_in_families['S_7247'] == 1))

mtxs.append(D)


E = mt.filter_cols(mt.phenotypes.family == 'E')
E = E.filter_rows((hl.agg.any(E.GT.is_non_ref())) &
                    ((E.variants_with_gts[('E','YES')]) == (E.cases_with_gts['E'])) &
                    (E.carriers_in_families['S_7252'] == 1) &
                    (E.carriers_in_families['S_7254'] == 0))
mtxs.append(E)


F = mt.filter_cols(mt.phenotypes.family == 'F')
F = F.filter_rows((hl.agg.any(F.GT.is_non_ref())) &
                          ((F.variants_with_gts[('F','YES')]) == (F.cases_with_gts['F'])) &
                          (F.carriers_in_families['WGS_147c'] == 0))

mtxs.append(F)

G = mt.filter_cols(mt.phenotypes.family == 'G')
G = G.filter_rows((hl.agg.any(G.GT.is_non_ref())) &
                          ((G.variants_with_gts[('G','YES')]) == (G.cases_with_gts['G'])))

mtxs.append(G)

H = mt.filter_cols(mt.phenotypes.family == 'H')
H = H.filter_rows((hl.agg.any(H.GT.is_non_ref())) &
                          ((H.variants_with_gts[('H','YES')]) == (H.cases_with_gts['H'])) &
                          (H.carriers_in_families['S_7274'] == 0))

mtxs.append(H)


I = mt.filter_cols(mt.phenotypes.family == 'I')
I = I.filter_rows((hl.agg.any(I.GT.is_non_ref())) &
                    ((I.variants_with_gts[('I','YES')]) == (I.cases_with_gts['I'])) &
                    (I.carriers_in_families['S_7291'] == 1))

mtxs.append(I)

J = mt.filter_cols(mt.phenotypes.family == 'J')
J = J.filter_rows((hl.agg.any(J.GT.is_non_ref())) &
                    ((J.variants_with_gts[('J','YES')]) == (J.cases_with_gts['J'])) &
                    (J.carriers_in_families['S_7307'] == 0))

mtxs.append(J)


R = mt.filter_cols(mt.phenotypes.family == 'R')
R = R.filter_rows((hl.agg.any(R.GT.is_non_ref())) &
                    ((R.variants_with_gts[('R','YES')]) == (R.cases_with_gts['R'])) &
                    (R.carriers_in_families['WGS_180b'] == 0))
mtxs.append(R)


S = mt.filter_cols(mt.phenotypes.family == 'S')
S = S.filter_rows((hl.agg.any(S.GT.is_non_ref())) &
                    ((S.variants_with_gts[('S','YES')]) == (S.cases_with_gts['S'])) &
                    (S.carriers_in_families['WGS_185c'] == 0) &
                    (S.carriers_in_families['WGS_6819'] == 0))
mtxs.append(S)



T = mt.filter_cols(mt.phenotypes.family == 'T')
T = T.filter_rows((hl.agg.any(T.GT.is_non_ref())) &
                    (T.carriers_in_families['WGS_6812'] == 0) &
                    (T.carriers_in_families['WGS_6814'] == 0) &
                    (T.carriers_in_families['WGS_D197a'] == 1) &
                    (T.carriers_in_families['WGS_D197b'] == 0) &
                    (T.carriers_in_families['WGS_D197c'] == 1) &
                    (T.carriers_in_families['WGS_D6811'] == 1) &
                    (T.carriers_in_families['WGS_D6813'] == 0) &
                    (T.carriers_in_families['WGS_D6815'] == 0) &
                    (T.carriers_in_families['WGS_D6816'] == 1))

    
mtxs.append(T)


U = mt.filter_cols(mt.phenotypes.family == 'U')
U = U.filter_rows((hl.agg.any(U.GT.is_non_ref())) &
                    ((U.variants_with_gts[('U','YES')]) == (U.cases_with_gts['U'])) &
                    (U.carriers_in_families['462'] == 0))

    
mtxs.append(U)


W = mt.filter_cols(mt.phenotypes.family == 'W')
W = W.filter_rows((hl.agg.any(W.GT.is_non_ref())) &
                    ((W.variants_with_gts[('W','YES')]) == (W.cases_with_gts['W'])) &
                    (W.carriers_in_families['476'] == 1) &
                    (W.carriers_in_families['479'] == 0) &
                    (W.carriers_in_families['468'] == 0))

    
mtxs.append(W)

                  
                  
X1 = mt.filter_cols(mt.phenotypes.family == 'X')
X1 = X1.filter_rows((hl.agg.any(X1.GT.is_non_ref())) &
                    (X1.carriers_in_families['480'] == 1) &
                    (X1.carriers_in_families['483'] == 1) &
                    (X1.carriers_in_families['481'] == 1) &
                    (X1.carriers_in_families['482'] == 1) &
                    (X1.carriers_in_families['484'] == 0))

    
mtxs.append(X1)
                    
                    
X2 = mt.filter_cols(mt.phenotypes.family == 'X')
X2 = X2.filter_rows((hl.agg.any(X2.GT.is_non_ref())) &
                    (X2.carriers_in_families['480'] == 1) &
                    (X2.carriers_in_families['484'] == 1) &
                    (X2.carriers_in_families['481'] == 0) &
                    (X2.carriers_in_families['482'] == 0) &
                    (X2.carriers_in_families['483'] == 0))

    
mtxs.append(X2)
                  
                  
Y = mt.filter_cols(mt.phenotypes.family == 'Y')
Y = Y.filter_rows((hl.agg.any(Y.GT.is_non_ref())) &
                    ((Y.variants_with_gts[('Y','YES')]) == (Y.cases_with_gts['Y'])) &
                    (Y.carriers_in_families['490'] == 0) &
                    (Y.carriers_in_families['492'] == 0))

    
mtxs.append(Y)

In [162]:
mtxs_0001 = []

for mtx in mtxs:
    mtx = mtx.filter_rows(mtx.gnomad_v3.v3_nfe.AF < 0.0001)
    mtxs_0001.append(mtx)

In [163]:
tbs = []

colss = []

for mtx in mtxs_0001:
    tb = mtx.select_cols(mtx.alt_id, mtx.phenotypes.family, mtx.phenotypes.disease, mtx.phenotypes.sex, mtx.phenotypes.kinship, mtx.phenotypes.add_pheno,
                        mtx.phenotypes.heavy_tics)
    tb = tb.select_entries(tb.GT, tb.GQ)
    tb = tb.select_rows(tb.rsid, tb.gnomad_v3.v3_nfe.AF, tb.gnomad_v3.v3_nfe.homozygote_count,
                        tb.gnomad_v3['most_severe_consequence'],
                        tb.within_gene, tb.hpo, tb.sporadic_cases, tb.variants_in_familial_controls, tb.variants_in_familial_cases)
    cols = tb.cols()
    colss.append(cols.to_pandas())
    
    tb = tb.make_table()
    tbs.append(tb.to_pandas())

2020-08-10 15:25:06 Hail: INFO: Coerced sorted dataset
2020-08-10 15:26:47 Hail: INFO: Coerced sorted dataset
2020-08-10 15:27:58 Hail: INFO: Coerced sorted dataset
2020-08-10 15:28:37 Hail: INFO: Coerced sorted dataset
2020-08-10 15:29:06 Hail: INFO: Coerced sorted dataset
2020-08-10 15:29:29 Hail: INFO: Coerced sorted dataset
2020-08-10 15:29:54 Hail: INFO: Coerced sorted dataset
2020-08-10 15:30:19 Hail: INFO: Coerced sorted dataset
2020-08-10 15:30:37 Hail: INFO: Coerced sorted dataset
2020-08-10 15:30:56 Hail: INFO: Coerced sorted dataset
2020-08-10 15:31:13 Hail: INFO: Coerced sorted dataset
2020-08-10 15:31:30 Hail: INFO: Coerced sorted dataset
2020-08-10 15:31:37 Hail: INFO: Coerced sorted dataset
2020-08-10 15:31:43 Hail: INFO: Coerced sorted dataset
2020-08-10 15:32:02 Hail: INFO: Coerced sorted dataset
2020-08-10 15:32:09 Hail: INFO: Coerced sorted dataset
2020-08-10 15:32:15 Hail: INFO: Coerced sorted dataset
2020-08-10 15:32:22 Hail: INFO: Coerced sorted dataset


In [170]:
for idx, (col, tb) in enumerate(zip(colss, tbs)):
    name = fams[idx]

    col_name = 'cols_'+str(name)   
    row_name = 'rows_'+str(name)

    col.to_pickle('/net/archive/groups/plggneuromol/GTS-analysis/py_objects/'+col_name)
    tb.to_pickle('/net/archive/groups/plggneuromol/GTS-analysis/py_objects/'+row_name)

In [171]:
cols = !ls /net/archive/groups/plggneuromol/GTS-analysis/py_objects/cols*
rows = !ls /net/archive/groups/plggneuromol/GTS-analysis/py_objects/rows*

info = []
tables = []


for col, row in zip(cols, rows):
    info.append(pd.read_pickle(col))
    tables.append(pd.read_pickle(row))

results = []

In [172]:
for idx, table in enumerate(tables):
    
    a = list(table.columns)

    b = [x for x in a if 'phased' not in x]
    b = [x for x in b if 'GQ' not in x]
    
    table = table[b]
    table = table.astype(str)
    
    tables[idx] = table
    
    e = pd.DataFrame(table.columns).transpose()
    e.columns = table.columns
    f = pd.concat([e, table], axis=0)

    g = info[idx].transpose()
    h = pd.DataFrame(g.index)
    h.index = g.index
    i = pd.concat([h, g], axis = 1)
    
    i.columns = range(0,len(i.columns))
    i = i.reindex(columns = ['a']*11 + list(range(0,len(i.columns))))
    
    i.columns = f.columns
    i.index = range(0,len(i.index))

    f.index = range(len(i.index)+1,len(i.index)+len(f.index)+1)

    j = pd.concat([i, f], axis = 0)
    j.columns = range(0, len(j.columns))
    
    
    for z in [9,10,11]:
        j[z].replace(regex=True,inplace=True,to_replace={'\{','\}','\[','\"','\]','key:','value:','value'},value='')
    
        j[z].replace(regex=True,inplace=True,to_replace='false,',value='refs:')
        j[z].replace(regex=True,inplace=True,to_replace='true,',value='non_refs:')
        j[z].replace(regex=True,inplace=True,to_replace='null,',value='not_called')
        j[z].replace(regex=True,inplace=True,to_replace='::',value=':')
    
    results.append(j)
    

In [174]:
with pd.ExcelWriter('families.xlsx') as writer:
    for idx, res in enumerate(results):
        res.to_excel(writer, sheet_name=str(fams[idx]), header=False, index=False)